In [ ]:
import SAGA
from easyquery import Query, QueryMaker
from SAGA.database import FitsTable
from SAGA import ObjectCuts as C
from SAGA import HostCuts as H
from astropy.table import unique

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

saga_cur = SAGA.QuickStart()

In [ ]:
specs_1 = FitsTable(saga._shared_dir + "/Paper1/saga_spectra_clean.fits.gz").read()

specs_1["HOSTID"] = np.char.add("nsa", specs_1["HOST_NSAID"].astype("<7U"))
specs_1["r_mag"] = specs_1["r"] - specs_1["EXTINCTION_R"]
specs_1["Mr"] = specs_1["r_mag"] - np.log10(specs_1["HOST_DIST"])*5 - 25

assert len(np.unique(specs_1["HOSTID"])) == 74

specs_1 = Query(saga.host_catalog.construct_host_query("paper1_complete"), C.is_galaxy).filter(specs_1)
has_our_specs_only_p1 = (QueryMaker.contains("SPEC_REPEAT", "IMACS") | QueryMaker.contains("SPEC_REPEAT", "WIYN") | C.has_our_specs_only)
print(has_our_specs_only_p1.count(specs_1))

In [ ]:
def remove_duplicated(specs):
    specs["SATS_sort"] = np.where(specs["SATS"] < 1, 999, specs["SATS"])
    specs.sort("SATS_sort")
    del specs["SATS_sort"]
    specs = unique(specs, "OBJID")
    specs.sort("HOSTID")
    return specs

In [ ]:
specs_2 = saga.object_catalog.load_clean_specs()
specs_2 = Query(C.is_clean2, C.is_galaxy2, saga.host_catalog.construct_host_query("paper2_complete")).filter(specs_2)
specs_2 = remove_duplicated(specs_2)

In [ ]:
hostlist = saga_cur.host_catalog.load("good", add_coord=False, include_stats=True)
sorter = (hostlist["paper2_need_spec"] / hostlist["paper2_total"]).argsort()
paper3_planned = QueryMaker.in1d("HOSTID", hostlist["HOSTID"][sorter[:100]])

In [ ]:
specs_3 = saga_cur.object_catalog.load_clean_specs()
specs_3 = Query(C.is_clean2, C.is_galaxy2, paper3_planned).filter(specs_3)
specs_3 = remove_duplicated(specs_3)

In [ ]:
q = Query(C.paper2_targeting_cut, C.basic_cut2)
low_z = Query("SPEC_Z >= 0.003", "SPEC_Z < 0.03")

ratio_sf = Query(q, C.is_sat, ~C.sdss_limit).count(specs_2) / Query(q, ~C.sdss_limit).count(specs_2) 
ratio_sb = Query(q, C.is_sat, C.sdss_limit).count(specs_2) / Query(q, C.sdss_limit).count(specs_2)

ratio_lf = Query(q, low_z, ~C.is_sat, ~C.sdss_limit).count(specs_2) / Query(q, ~C.sdss_limit).count(specs_2) 
ratio_lb = Query(q, low_z, ~C.is_sat,  C.sdss_limit).count(specs_2) / Query(q, C.sdss_limit).count(specs_2)

In [ ]:
X = []

for i, specs in enumerate([
    C.sat_rcut.filter(specs_1), 
    specs_1, 
    C.sat_rcut.filter(specs_2), 
    specs_2, 
    C.sat_rcut.filter(specs_3),
    specs_3,
]):
    if i < 2:
        has_our_specs_only = has_our_specs_only_p1
    else:
        has_our_specs_only = C.has_our_specs_only
    
    if i <= 1: 
        paper1_fix = -2
    else:
        paper1_fix = 0
        
    if i >= 4:
        paper3_new = Query(paper3_planned, ~saga_cur.host_catalog.construct_host_query("paper2_complete"))
        spec_funge = paper3_new.filter(hostlist, "paper2_need_spec").sum() + (50 * paper3_new.count(hostlist))
        nb = paper3_new.filter(hostlist, "really_need_spec_bright").sum()
        nf = paper3_new.filter(hostlist, "paper2_need_spec").sum() - nb
        sat_funge = int((nb * ratio_sb + nf * ratio_sf))
        lowz_funge = int((nb * ratio_lb + nf * ratio_lf))
        if i == 5:
            spec_funge += int(85 * paper3_new.count(hostlist))
            lowz_funge += int(3 * paper3_new.count(hostlist))
            
        def rounding(x):
            if x >= 10000:
                step = 1000
            elif x >= 1000:
                step = 100
            else:
                step = 50
            return int(np.round(x / step)*step)
    else:
        spec_funge = 0
        sat_funge = 0
        lowz_funge = 0
        rounding = lambda x: x
        
    X.append((
        len(np.unique(specs["HOSTID"])),
        rounding(has_our_specs_only.count(specs) + spec_funge + paper1_fix),
        rounding(len(specs) + spec_funge),
        rounding(Query(C.is_sat, has_our_specs_only, C.r_abs_limit).count(specs) + sat_funge),
        rounding(Query(C.is_sat, has_our_specs_only, ~C.r_abs_limit).count(specs)),
        rounding(Query(C.is_sat, C.r_abs_limit).count(specs) + sat_funge),
        rounding(Query(C.is_sat, ~C.r_abs_limit).count(specs)),
        rounding(Query(low_z, has_our_specs_only).count(specs) + sat_funge + lowz_funge),
        rounding(Query(low_z).count(specs) + sat_funge + lowz_funge),
    ))
    
X = np.array(X).T
X

In [ ]:
labels = [r"\# Complete Hosts", r"\# Galaxy Redshifts SAGA Obtained", r"Total \# Galaxy Redshifts", 
          "", r"\# Satellites Discovered",
          "", r"Total \# Satellites",
          r"\# Low-$z$ Galaxies Discovered", r"Total \# Low-$z$ Galaxies"]

sim = ["", "", r"$\sim$\,"]

def print_row(label, out_iter, nline):
    ending = "" if nline is None else (r"\\ " + (r"\hline" * nline))
    print(label, "&", " & ".join(out_iter), ending)

for i, (label, row) in enumerate(zip(labels, X)):
    row = ["{:,}".format(r) for j, r in enumerate(row)]
    
    if i == 0:
        print_row(label, (f"\multicolumn{{2}}{{c|}}{{{item}}}" for item in row[::2]), 2)
        print_row("Field Definition", (r"$\mathbf{<300}$\,\textbf{kpc} & $<1^{\circ}$" for _ in range (3)), 1)
    elif i in (3, 5):
        hold = np.array(row)
    elif i in (4, 6):
        out_tier = (fr"{s}\textbf{{{item} ({i2})}} & \ditto" for item, i2, s in zip(hold[::2], row[::2], sim))
        print_row(label, out_tier, 1)
    else:
        out_tier = (fr"{s}\textbf{{{i1}}} & {s}{i2}" for i1, i2, s in zip(row[::2], row[1::2], sim))
        print_row(label, out_tier, 1 if (i < len(X) -1) else None)

In [ ]:
p2_hostlist = saga.host_catalog.load(include_stats=True, query="paper2_complete")

numbers_dict = {
    "nhosts": len(p2_hostlist),
    "nhostsnew": (~H.paper1_complete).count(p2_hostlist), 
    "nhoststotal": len(saga.host_catalog.load("good")), 
    "nhostsinclnoimage": len(saga.host_catalog.load("good_hosts")), 
    "nhostswsats": Query("sats_total > 0").count(p2_hostlist), 
    "nhostswsatslimit": Query("sats_Mr_limit > 0").count(p2_hostlist), 
    "nhostsnosats": Query("sats_total == 0").count(p2_hostlist), 
    "nhostsnosatslimit": Query("sats_Mr_limit == 0").count(p2_hostlist), 

    "nsats": X[5, 2] + X[6, 2], 
    "nsatslimit": X[5, 2], 
    "nsatsbelow": X[6, 2], 
    "nsatssaga": X[3, 2] + X[4, 2], 
    "nsatslimitsaga": X[3, 2], 
    "nsatsother": X[5, 2] + X[6, 2] - X[3, 2] - X[4, 2], 
    "nsatslimitother": X[5, 2] - X[3, 2], 

    "nzsaga": X[1, 3], 
    "nztotal": X[2, 3], 
    "nzsaganew": X[1, 3] - X[1, 1],
    "nztotalnew": X[2, 3] - X[2, 1], 
    "nzother": X[2, 3] - X[1, 3], 
    "nzothernew": (X[2, 3] - X[1, 3]) - (X[2, 1] - X[1, 1]), 

    "nzsagarvir": X[1, 2], 
    "nztotalrvir": X[2, 2],  
    "nzsaganewrvir": X[1, 2] - X[1, 0],
    "nztotalnewrvir": X[2, 2] - X[2, 0], 
    "nzotherrvir": X[2, 2] - X[1, 2], 
    "nzothernewrvir": (X[2, 2] - X[1, 2]) - (X[2, 0] - X[1, 0]),
}

In [ ]:
for k, n in numbers_dict.items():
    print("\\newcommand*{{\\{}}}{{{:,}}}".format(k, n))